In [ ]:
# for plotting

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

sns.set()
sns.set_context("poster")
sns.set_style("ticks")
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "cmr10"
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
import numpy as np
import xarray as xr
import metpy
from metpy.units import units
from metpy.constants import water_heat_vaporization, dry_air_gas_constant, earth_gravity
from metpy.interpolate import interpolate_1d
from pandas import Timestamp, Timedelta
import netCDF4 as nc
import glob

In [ ]:
def extract_sounding_sonde_pin(sonde_file):
    sonde = xr.open_dataset(sonde_file)
    t = sonde.tdry + 273.15  # deg C to K
    p = sonde.pres * 100.0
    rv = metpy.calc.mixing_ratio_from_relative_humidity(
        sonde.pres.metpy.quantify(),
        sonde.tdry.metpy.magnitude * units("degC"),
        sonde.rh.metpy.quantify(),
    )
    z = sonde.alt
    u = sonde.u_wind
    v = sonde.v_wind
    return [z, p, t, rv, u, v]

In [ ]:
def extract_sfc_fluxes_era5_pin(d, t1, t2, lat, lon, dx=2.5):
    g = earth_gravity
    shf = d.ishf.loc[t1:t2, lat + dx : lat - dx, lon - dx : lon + dx].mean(axis=(1, 2))
    lhf = (
        (water_heat_vaporization * d.ie.metpy.quantify())
        .loc[t1:t2, lat + dx : lat - dx, lon - dx : lon + dx]
        .mean(axis=(1, 2))
    )
    sst = d.sst.loc[t1:t2, lat + dx : lat - dx, lon - dx : lon + dx].mean(axis=(1, 2))
    lsm = d.lsm.loc[t1, lat + dx : lat - dx, lon - dx : lon + dx].max()
    print(
        "Latitude points in the averageing box: "
        + repr(d.latitude.loc[lat + dx : lat - dx].values.tolist())
    )
    print(
        "Longitude points in the averageing box: "
        + repr(d.longitude.loc[lon - dx : lon + dx].values.tolist())
    )
    time = [(t - t1).total_seconds() for t in d.time.loc[t1:t2].values]
    zs = (
        d.z.loc[t1:t2, lat + dx : lat - dx, lon - dx : lon + dx]
        .metpy.quantify()
        .mean(axis=(1, 2))
    )
    # print(d.z.loc[t1, lat+dx:lat-dx, lon-dx:lon+dx].values)
    if lsm > 0.5:
        # According to ECMWF documentation, lsm > 0.5 is considered ocean/water.
        # But actually none of the points around ENA has lsm = 0.0 either.
        print("Averaging area contains over-land grid points!")
    return np.asarray(time), sst, shf, lhf, zs.metpy.quantify() / g

In [ ]:
def extract_forcing_era5_pin(d, t1, t2, lat, lon, dx_in=2.5):
    """
    Extract large-scale subsidence, horizontal winds, geostrophic horizontal winds, large-scale horizontal
    advection tendencies of temperature and moisture from the forcing dataset.
    """
    # use a larger box to calculate things
    dx = dx_in * 5.0

    """
    interpolation to pressure levels and calculate geopotential heights following:
    https://confluence.ecmwf.int/display/CKB/ERA5%3A+compute+pressure+and+geopotential+on+model+levels%2C+geopotential+height+and+geometric+height
    """
    rd = dry_air_gas_constant
    g = earth_gravity
    ab = np.genfromtxt(
        "era5_table.csv", delimiter=",", skip_header=1, missing_values="-"
    )
    a = ab[:, 1]
    b = ab[:, 2]

    t = d.t.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx]
    qv = d.q.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx]
    u = d.u.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx]
    v = d.v.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx]
    omega = d.w.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx]
    lnsp = d.lnsp.loc[t1:t2, 1, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx]
    # the surface geopotential looks so noisy because of the spectral decomposition/representation used in IFS
    sgp = d.z.loc[t1:t2, 1, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx]
    rv = metpy.calc.mixing_ratio_from_specific_humidity(qv)

    nt, nz, ny, nx = t.shape

    pi = np.zeros((nt, nz + 1, ny, nx))
    ps = np.exp(lnsp)
    pi[:] = ps.values[:, np.newaxis, :, :]
    pi = (
        a[np.newaxis, :, np.newaxis, np.newaxis]
        + pi * b[np.newaxis, :, np.newaxis, np.newaxis]
    )
    p = (pi[:, 1:, :, :] + pi[:, :-1, :, :]) * 0.5
    pi[:, 0, :, :] = 0.1
    dpi = pi[:, 1:, :, :] - pi[:, :-1, :, :]
    dlnpi = np.log(pi[:, 1:, :, :] / pi[:, :-1, :, :])

    # have not got time to derive alpha, this is just what is given in the ERA5 documentation
    alpha = 1.0 - dlnpi * pi[:, :-1, :, :] / dpi
    alpha[:, 0, :, :] = np.log(2.0)

    tm = t.metpy.quantify() * (1.0 + 0.609133 * rv.metpy.quantify())
    # tm = t*(1.0+0.609133*qv)
    dphi = rd.magnitude * tm.values * dlnpi
    phi = np.zeros((nt, nz + 1, ny, nx))
    phi[:, :-1, :, :] = np.flip(np.cumsum(dphi[:, ::-1, :, :], axis=1), 1)
    phi[:] = phi[:] + sgp.values[:, np.newaxis, :, :]
    ph = phi[:, 1:, :, :] + rd.magnitude * tm.values * alpha

    ph = t.copy(deep=True, data=ph)
    ph.attrs["units"] = "m**2/s**2"
    del ph.attrs["long_name"]
    del ph.attrs["standard_name"]
    ph.metpy.quantify()

    p = t.copy(deep=True, data=p)
    p.attrs["units"] = "Pa"
    del p.attrs["long_name"]
    del p.attrs["standard_name"]
    p.metpy.quantify()

    z = ph.metpy.quantify() / g
    z = t.copy(deep=True, data=z)
    z.attrs["units"] = "m"
    del z.attrs["long_name"]
    del z.attrs["standard_name"]

    w = -omega.metpy.quantify() * rd * tm.metpy.quantify() / p.metpy.quantify() / g
    # print(omega.metpy.units)
    # print(tm.metpy.units)
    # print(rd.units)
    # print(p.metpy.units)
    # print(w.metpy.units)

    ph = ph.metpy.assign_crs(
        grid_mapping_name="latitude_longitude", earth_radius=6371229.0
    )
    p = p.metpy.assign_crs(
        grid_mapping_name="latitude_longitude", earth_radius=6371229.0
    )
    z = z.metpy.assign_crs(
        grid_mapping_name="latitude_longitude", earth_radius=6371229.0
    )

    # calculate geostrophic winds
    lnp = np.log(p)
    lnp.attrs["units"] = ""
    lnp.metpy.quantify()
    dpx1, dpy1 = metpy.calc.geospatial_gradient(ph)
    dpx2, dpy2 = metpy.calc.geospatial_gradient(lnp)
    dpx = dpx1 + rd * tm.metpy.quantify() * dpx2
    dpy = dpy1 + rd * tm.metpy.quantify() * dpy2
    f = metpy.calc.coriolis_parameter(lat * units("degrees"))
    vg = dpx / f
    ug = -dpy / f

    # calculate horizontal advection tendencies
    # tp = calc_tp_era5(d)
    tp = metpy.calc.potential_temperature(p, t)
    ma = metpy.calc.advection(rv, u, v)
    ta = metpy.calc.advection(tp, u, v)

    time = [(t - t1).total_seconds() for t in d.time.loc[t1:t2].values]

    # output just points within the specified box
    dx = dx_in
    print(
        "Latitude points in the averageing box: "
        + repr(d.latitude.loc[lat + dx : lat - dx].values.tolist())
    )
    print(
        "Longitude points in the averageing box: "
        + repr(d.longitude.loc[lon + 360.0 - dx : lon + 360.0 + dx].values.tolist())
    )
    return (
        np.asarray(time),
        d.z.loc[
            t1:t2, 1, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx
        ].metpy.quantify()
        / g,
        ps.loc[t1:t2, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
        z.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
        p.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
        w.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
        d.w.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
        d.u.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
        d.v.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
        ta.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
        ma.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
        ug.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
        vg.loc[t1:t2, :, lat + dx : lat - dx, lon + 360 - dx : lon + 360 + dx],
    )

In [ ]:
def interpolate_forcing_era5(zout, z, w, u, v, ta, ma, ug, vg, zs):
    """
    interpolate all the forcing profiles to SAM's vertical grid given by `zout`
    """

    nt, nz, ny, nx = z.shape
    zi = np.zeros((nt, nz + 1, ny, nx))
    """
    This is just one way to take into account surface elevation.
    The other way is to interpolate using z (heights above MSL) directly,
    then add a mean surface elevation to zout. Using the latter method, the averaging
    will be done on heights above MSL rather than heights above surface.
    But given the small surface elevations we have around ENA, it shouldn't make much difference. 
    """
    zi[:, :-1, :, :] = z.values - zs.values[:, np.newaxis, :, :]
    # zi[:, :-1, :, :] = z.values
    # zout = zout + zs.values.mean()
    out = []
    sfc = [0, 0, 0, 1, 1, 1, 1]
    for v, s in zip([w, u, v, ta, ma, ug, vg], sfc):
        vi = np.zeros_like(zi)
        vi[:, :-1, :, :] = v.values
        # surface winds are all zero
        if s == 1:
            vi[:, -1, :, :] = v.values[:, -1, :, :]
        # vi[:, -1, :, :] = v.values[:, -1, :, :]
        vo = interpolate_1d(zout, zi, vi, axis=1)
        out.append(vo.mean(axis=(2, 3)))
    return out

In [ ]:
zs_snd = 30.0  # surface elevation of ENA site in m
domain_height = 6500.0  # model top
dz_out = 25.0  # m
z_out = np.arange(dz_out * 0.5, domain_height + dz_out, dz_out)
lat = 39.0916  # deg N
lon = -28.0257  # deg E
lon_w = - lon  # deg W, used by the solar zenith angle calculation method in the RRTM wrapper
forc_dir = "/ccsopen/home/hengxiao80/proj/ena_forcing_check/forcing"
sonde_dir = "/ccsopen/home/hengxiao80/proj/ena_forcing_check/obs/enasondewnpnC1"
# start_time = Timestamp("2016-10-22 00:00:00")
# end_time = Timestamp("2016-10-23 00:00:00")
# start_time = Timestamp("2018-11-21 00:00:00")
# end_time = Timestamp("2018-11-22 00:00:00")
start_time = Timestamp("2018-10-28 00:00:00")
end_time = Timestamp("2018-10-29 00:00:00")
start_doy = start_time.dayofyear
start_hour = start_time.hour

In [ ]:
outdate = start_time.strftime("%Y%m%d_%H%M")
outfile = f"{outdate}.era5.nc"
outroot = nc.Dataset(outfile, "w")

In [ ]:
snd_time_adj = start_time - Timedelta(1, "h")
snd_datestring = snd_time_adj.strftime("%Y%m%d.%H")
snd_file = glob.glob(f"{sonde_dir}/enasondewnpnC1.b1.{snd_datestring}*.cdf")[0]
z_snd, p_snd, t_snd, rv_snd, u_wind_snd, v_wind_snd = extract_sounding_sonde_pin(
    snd_file
)

In [ ]:
init_group = outroot.createGroup("initialization")
init_group.createDimension("z", len(z_snd))
init_group.createDimension("const", 1)

surfp = init_group.createVariable("surface_pressure", "f8", ("const"))
surfp[:] = p_snd[0]  # p_srf
surft = init_group.createVariable("surface_temperature", "f8", ("const"))
surft[:] = t_snd[0]  # t_srf

u_in_domain = u_wind_snd[z_snd < domain_height]
u00 = 0.5 * (np.amin(u_in_domain) + np.amax(u_in_domain))
v_in_domain = v_wind_snd[z_snd < domain_height]
v00 = 0.5 * (np.amin(v_in_domain) + np.amax(v_in_domain))
u0 = init_group.createVariable("reference_u0", "f8", ("const"))
u0[:] = u00
v0 = init_group.createVariable("reference_v0", "f8", ("const"))
v0[:] = v00

z_ini = init_group.createVariable("z", "f8", ("z"))
z_ini[:] = z_snd[:] - zs_snd
pres_ini = init_group.createVariable("pressure", "f8", ("z"))
pres_ini[:] = p_snd[:]
t_ini = init_group.createVariable("temperature", "f8", ("z"))
t_ini[:] = t_snd[:]
qv_ini = init_group.createVariable("vapor_mixing_ratio", "f8", ("z"))
qv_ini[:] = rv_snd.magnitude[:]
u_ini = init_group.createVariable("u", "f8", ("z"))
u_ini[:] = u_wind_snd[:]
v_ini = init_group.createVariable("v", "f8", ("z"))
v_ini[:] = v_wind_snd[:]

In [ ]:
datem1 = (start_time - Timedelta(1, "D")).strftime("%Y%m%d")
datep1 = (start_time + Timedelta(1, "D")).strftime("%Y%m%d")
era5_sfc = xr.open_dataset(f"{forc_dir}/era5/data/ERA5-{datem1}-{datep1}-sfc.nc")
era5_sfc = era5_sfc.sortby(era5_sfc.time)
era5_l = xr.open_dataset(f"{forc_dir}/era5/data/ERA5-{datem1}-{datep1}-ml.nc")

In [ ]:
time_sfc_era5, sst_era5, shf_era5, lhf_era5, zs_sfc_era5 = extract_sfc_fluxes_era5_pin(
    era5_sfc, start_time, end_time, lat, lon
)

In [ ]:
sfc = outroot.createGroup("surface")
sfc.createDimension("t", len(time_sfc_era5))

time_sfc = sfc.createVariable("times", "f8", ("t"))
time_sfc[:] = time_sfc_era5[:]

shf = sfc.createVariable("sensible_heat_flux", "f8", ("t"))
shf[:] = - shf_era5[:]
lhf = sfc.createVariable("latent_heat_flux", "f8", ("t"))
lhf[:] = - lhf_era5[:]
tsk = sfc.createVariable("skin_temperature", "f8", ("t"))
tsk[:] = sst_era5[:]
ustar = sfc.createVariable("friction_velocity", "f8", ("t"))
ustar[:] = np.ones(len(time_sfc_era5)) * 0.25

In [ ]:
time_era5, zs, ps, z, p, w, omega, u, v, ta, ma, ug, vg = extract_forcing_era5_pin(
    era5_l, start_time, end_time, lat, lon
)
(
    w_era5,
    u_era5,
    v_era5,
    ta_era5,
    ma_era5,
    ug_era5,
    vg_era5,
) = interpolate_forcing_era5(z_out, z, w, u, v, ta, ma, ug, vg, zs)
ps_era5 = ps.mean(axis=(1, 2))
del zs, ps, z, p, w, omega, u, v, ta, ma, ug, vg

In [ ]:
forcing_group = outroot.createGroup("forcing")

forcing_group.createDimension("t", len(time_era5))
forcing_group.createDimension("z", len(z_out))
forcing_group.createDimension("const", 1)

time_lsf = forcing_group.createVariable("times", "f8", ("t"))
time_lsf[:] = time_era5[:]
z_lsf = forcing_group.createVariable("z", "f8", ("z"))
z_lsf[:] = z_out[:]
lat_lsf = forcing_group.createVariable("latitude", "f8", ("const"))
lat_lsf[:] = lat

ug_lsf = forcing_group.createVariable("u_geostrophic", "f8", ("t", "z"))
ug_lsf[:, :] = ug_era5[:, :]
vg_lsf = forcing_group.createVariable("v_geostrophic", "f8", ("t", "z"))
vg_lsf[:, :] = vg_era5[:, :]
ur_lsf = forcing_group.createVariable("u_relaxation", "f8", ("t", "z"))
ur_lsf[:, :] = u_era5[:, :]
vr_lsf = forcing_group.createVariable("v_relaxation", "f8", ("t", "z"))
vr_lsf[:, :] = v_era5[:, :]
w_lsf = forcing_group.createVariable("subsidence", "f8", ("t", "z"))
w_lsf[:, :] = w_era5[:, :]
tha_lsf = forcing_group.createVariable("theta_advection", "f8", ("t", "z"))
tha_lsf[:, :] = ta_era5[:, :]
qva_lsf = forcing_group.createVariable("qt_advection", "f8", ("t", "z"))
qva_lsf[:, :] = ma_era5[:, :]
# not doing relaxation for thermodynamics for now
t_lsf = forcing_group.createVariable("th_relaxation", "f8", ("t", "z"))
t_lsf[:, :] = 0.0
qv_lsf = forcing_group.createVariable("qt_relaxation", "f8", ("t", "z"))
qv_lsf[:, :] = 0.0

In [ ]:
# Radiation profile (z,p) set in same way as PINACLES reference

h_rad = np.logspace(3.5, 4.3)
p_rad = np.interp(h_rad, z_snd - zs_snd, p_snd)
# print(p_rad)
# print(h_rad)
# plt.plot(p_rad, h_rad, "o")

In [ ]:
# coszrs = 1.0
# aldir = (0.026 / (coszrs**1.7 + 0.065)) + (
#     0.15 * (coszrs - 0.10) * (coszrs - 0.50) * (coszrs - 1.00)
# )
# print(aldir)

In [ ]:
rad_group = outroot.createGroup("radiation")
rad_group.createDimension("layers", len(h_rad))
rad_group.createDimension("const", 1)

radlat = rad_group.createVariable("latitude", "f8", ("const"))
radlat[:] = lat * 1.0
radlon = rad_group.createVariable("longitude", "f8", ("const"))
radlon[:] = lon_w * 1.0
day_year = rad_group.createVariable("day_of_year", "f8", ("const"))
day_year[:] = start_doy * 1.0
hour_utc = rad_group.createVariable("hour_utc", "f8", ("const"))
hour_utc[:] = start_hour * 1.0
emissivity = rad_group.createVariable("emissivity", "f8", ("const"))
emissivity[:] = 0.95 # Value from SAM RRTMG
albedo = rad_group.createVariable("albedo", "f8", ("const"))
albedo[:] = 0.06 # Value for diffusive albedeo in SAM RRTMG
h = rad_group.createVariable("z", "f8", ("layers"))
h[:] = h_rad[:]
p = rad_group.createVariable("pressure", "f8", ("layers"))
p[:] = p_rad[:]
t = rad_group.createVariable("temperature", "f8", ("layers"))
t[:] = np.interp(h_rad, z_snd - zs_snd, t_snd)
qv = rad_group.createVariable("vapor_mixing_ratio", "f8", ("layers"))
qv[:] = np.interp(h_rad, z_snd - zs_snd, rv_snd.magnitude)
ql = rad_group.createVariable("liquid_mixing_ratio", "f8", ("layers"))
ql[:] = np.zeros(h_rad.shape)
qi = rad_group.createVariable("ice_mixing_ratio", "f8", ("layers"))
qi[:] = np.zeros(h_rad.shape)

In [ ]:
outroot.close()